## RIS-Export von Gartenlaube-Artikeln zum Deutsch-Französischen Krieg 1870/71

Das ab 1853 erschienene illustrierte Familienmagazin [Die Gartenlaube](https://de.wikipedia.org/wiki/Die_Gartenlaube) hat nicht zuletzt aufgrund des zeitlichen Zusammenfalls zwischen dem eigenen Erscheinungszeitraum und dem Ereignis des Deutsch-Französischen Krieges 1870/71 über dieses Geschehen berichtet.

Für die offene [Zotero-Bibliographie](https://www.zotero.org/groups/2481783/guerre-krieg_187071) herausgegeben von Mareike Köng (DHIP) und dem Social-Mediaprojekt [Guerre1870](https://guerre1870.hypotheses.org/2381) trägt die Datenlaube per simplen Export der Gartenlaube-Artikel zum Thema einen Beitrag zum Ausbau dieses Bibliographie. 

Das folgende Python-Skript erledigt diesen Export:

1. SPARQL-Abfrage der Gartenlaube-Artikel die den [Deutsch-Französischen Krieg](https://www.wikidata.org/wiki/Q46083) zum Thema haben.
2. Transformation von Titel, Autor, Publikationsjahr, Heftnummer, Seite, Wikisource-Link (Volltext) und Schlagwörter in das [RIS-Format](https://de.wikipedia.org/wiki/RIS_(Dateiformat))
3. RIS-Text kann per Copy/Paste in Zotero in die Bibliographie integriert werden. [Gartenlaube-Quellen in der Bibliographie](https://www.zotero.org/groups/2481783/guerre-krieg_187071/collections/N3ECZG8E)



In [1]:
#!pip install sparqlwrapper
#https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT DISTINCT ?item ?title ?hefte ?authorLabel ?authorNameStr ?pubyear  ?seiteN ?wsPage  (GROUP_CONCAT(?schlagwortLabel;separator=" ; ") AS ?swf) WITH {
SELECT  ?item ?author ?authorNameStr ?title ?pubString ?hefte ?pubyear ?wsPage ?schlagwort (GROUP_CONCAT(?seite_st;separator=";") AS ?seiteN) WHERE {
  ?item wdt:P1433 wd:Q655617;
    wdt:P921 wd:Q46083;
    wdt:P1476 ?title;
    wdt:P577 ?pubdate;
    p:P433 ?heftStmt;
    wdt:P304 ?seite_st.
  BIND(YEAR(?pubdate) AS ?pubyear)
  ?heftStmt ps:P433 ?hefte.
  OPTIONAL { ?heftStmt pq:P304 ?seite_q. }
  BIND(CONCAT("In: Die Gartenlaube (",STR(YEAR(?pubdate)),") Nr. ",?hefte," S. ") AS ?pubString)
  ?wsPage schema:about ?item;
          schema:isPartOf <https://de.wikisource.org/>.
  OPTIONAL { ?item wdt:P50 ?author. }
  OPTIONAL { ?item wdt:P2093 ?authorNameStr. }
  OPTIONAL { ?item wdt:P921 ?schlagwort. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
GROUP BY  ?item ?title ?author ?authorNameStr ?pubString ?hefte ?pubyear ?wsPage ?schlagwort
ORDER BY ?pubyear
                } AS %guerreBib
WHERE {include %guerreBib
            SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
           ?schlagwort rdfs:label ?schlagwortLabel. FILTER(LANG(?schlagwortLabel)="de")
  }
GROUP BY ?item ?title ?hefte ?authorLabel ?authorNameStr ?pubyear  ?seiteN ?wsPage  """


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    #print(result)
    print("TY - MGZN")
    print("JF - Die Gartenlaube")
    print("TI - "+result["title"]["value"])
    print("PY - "+result["pubyear"]["value"])
    print("SP - "+result["seiteN"]["value"])
    print("IS - "+result["hefte"]["value"])
    try:
        print("A1 - "+result["authorLabel"]["value"])
    except KeyError:
        try:
            print("A1 - "+result["authorNameStr"]["value"])
        except KeyError:
            pass
    
    print("UR - "+result["wsPage"]["value"])
    print("L2 - "+result["wsPage"]["value"])
    
    keywords = result["swf"]["value"].split(";")
    for kw in keywords:
        print("KW - "+kw.strip())
    
    print("ER -")



TY - MGZN
JF - Die Gartenlaube
TI - Ein Hauptquartier auf dem Marsch
PY - 1870
SP - 837–839;824–827
IS - 48–49
UR - https://de.wikisource.org/wiki/Ein_Hauptquartier_auf_dem_Marsch
L2 - https://de.wikisource.org/wiki/Ein_Hauptquartier_auf_dem_Marsch
KW - Schlacht bei Spichern
ER -
TY - MGZN
JF - Die Gartenlaube
TI - 1870
PY - 1880
SP - 529
IS - 33
A1 - Friedrich Hofmann
UR - https://de.wikisource.org/wiki/1870
L2 - https://de.wikisource.org/wiki/1870
KW - 1870
KW - Deutsch-Französischer Krieg
ER -
TY - MGZN
JF - Die Gartenlaube
TI - Ein guter und schöner Gedanke
PY - 1871
SP - 324
IS - 19
UR - https://de.wikisource.org/wiki/Ein_guter_und_sch%C3%B6ner_Gedanke
L2 - https://de.wikisource.org/wiki/Ein_guter_und_sch%C3%B6ner_Gedanke
KW - Deutsch-Französischer Krieg
KW - Waise
ER -
TY - MGZN
JF - Die Gartenlaube
TI - Zum Gedenktage von Sedan
PY - 1880
SP - 561
IS - 35
A1 - Ernst Ziel
UR - https://de.wikisource.org/wiki/Zum_Gedenktage_von_Sedan
L2 - https://de.wikisource.org/wiki/Zum_Gedenktag